Here I modify my code to make the slack bus selectable.

Recall that there are four components of an instance of the temporal instanton problem:

* $n$, $n_r$, and $T$

* $Q_x$, which depends on desired correlation

* $A$, which depends on admittance matrix $Y$

* $b$, which depends on $G^0$ and $D$

* $Q_\theta$, which depends on $\tau$ and the line we wish to overload

$Q_x$ does not depend on the choice of slack node, nor does $b$. $A$ does depend on which bus is chosen to be slack. Here is a modified method that accepts the index of the slack node as an argument and generates an A matrix:

In [46]:
function tmp_inst_A(n,Ridx,T,Y,slack,k)
    """ Generate the power balance constraint A matrix
    from problem dimensions, admittance matrix,
    and generator participation factors.
    Assumes the admittance matrix is n-by-n.
    
    Returns A, which is (n+1)*T-by-(n+nr)*T
    
    * n is the number of nodes in the network
    * Ridx is a vector indicating wind nodes
    * T is the number of time steps
    * Y is the admittance matrix (n-by-n)
    * slack is the index of the slack bus
    * k is the vector of generator participation factors
    """
    
    # zero out slack row of Y:
    adm = deepcopy(Y)
    adm[slack,:] = zeros(n)
    
    # remove slack column of Y:
    adm = adm[:,setdiff(1:n,slack)]
    
    # A has a block diagonal pattern where each
    # block is Atemp:
    Atemp = zeros(n+1,2*n)
    Atemp = [[-eye(n) adm -k]; ones(1,n) zeros(1,n-1) 1]
    
    # Remove columns corresponding to non-wind nodes
    # and slack node:
    Atemp = sparse(Atemp[:,[Ridx,n+1:2*n]])
    
    # Now we can tile the Atemp matrix to generate A:
    A = Atemp
    for t = 2:T
        A = blkdiag(A, Atemp)
    end
    
    return A
end

tmp_inst_A (generic function with 1 method)

In [50]:
# Test generateA and generateb:
n = 3
Ridx = [1,2,3]
T = 2
Y = [2 3 4;
    3 4 3;
    2 3 4]
slack = 3
k = [0.25,0.25,0.5]

A = tmp_inst_A(n,Ridx,T,Y,slack,k)
full(A)

8x12 Array{Float64,2}:
 -1.0   0.0   0.0  2.0  3.0  -0.25   0.0   0.0   0.0  0.0  0.0   0.0 
  0.0  -1.0   0.0  3.0  4.0  -0.25   0.0   0.0   0.0  0.0  0.0   0.0 
  0.0   0.0  -1.0  0.0  0.0  -0.5    0.0   0.0   0.0  0.0  0.0   0.0 
  1.0   1.0   1.0  0.0  0.0   1.0    0.0   0.0   0.0  0.0  0.0   0.0 
  0.0   0.0   0.0  0.0  0.0   0.0   -1.0   0.0   0.0  2.0  3.0  -0.25
  0.0   0.0   0.0  0.0  0.0   0.0    0.0  -1.0   0.0  3.0  4.0  -0.25
  0.0   0.0   0.0  0.0  0.0   0.0    0.0   0.0  -1.0  0.0  0.0  -0.5 
  0.0   0.0   0.0  0.0  0.0   0.0    1.0   1.0   1.0  0.0  0.0   1.0 

## Modified $Q_\theta$

If we allow the slack node to be passed as an argument, we must build $Q_\theta$ and remove the appropriate row and column afterward. The following method does this:

In [55]:
function tmp_inst_Qtheta(n,nr,T,tau,line,slack)
    """ Generate Q_theta in the temperature constraint
    of a temporal instanton problem instance.
    "line" has the form (i,k), where i and k refer to
    the endpoints of the chosen line.
    """
    Qtheta = zeros((n+nr)*T,(n+nr)*T)
    i,k = line
    ei = zeros(n,1)
    ei[i] = 1
    ek = zeros(n,1)
    ek[k] = 1
    
    Q0 = (ei - ek)*(ei - ek)'
    
    Q0 = Q0[:,setdiff(1:n,slack)]
    Q0 = Q0[setdiff(1:n,slack),:]
    
    for t = 1:T
        start = nr + 1 + (nr+n)*(t-1)
        stop = start + n - 2
        Qtheta[start:stop,start:stop] = tau^(T-t)*Q0
    end
    return sparse(Qtheta)
end

tmp_inst_Qtheta (generic function with 1 method)

In [86]:
n = 4
nr = 1
T = 1
tau = 2
line = (1,3)
k = [0.25,0.25,0.5]
slack = 1

Qtheta = tmp_inst_Qtheta(n,nr,T,tau,line,slack)
full(Qtheta)

5x5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

## Generating RTS-96 generation and demand data

Because we do not have time-coupled data for the RTS-96 network, we need to generate some.

I could take multiple hours from the ARPA data Jenny gave me.



In [87]:
include("../src/tmp_inst_rts96.jl")

readRTS96Data (generic function with 1 method)

In [89]:
psData = psDataLoad()
# unpack psDL (boilerplate):
(Sb,f,t,r,x,b,Y,bustype,
Gp,Gq,Dp,Dq,Rp,Rq,
Pmax,Pmin,Qmax,Qmin,Plim,
Vg,Vceiling,Vfloor,
busIdx,N,Nr,Ng,k) = unpack_psDL(psData)

(100.0,[1,1,1,2,2,3,3,4,5,6  …  66,66,67,67,68,68,69,73,66,71],[2,3,5,4,6,9,24,9,10,10  …  69,69,68,68,71,71,70,21,47,73],[0.003,0.055,0.022,0.033,0.05,0.031,0.002,0.027,0.023,0.014  …  0.003,0.003,0.005,0.005,0.003,0.003,0.009,0.012,0.013,0.0],[0.014,0.211,0.085,0.127,0.192,0.119,0.084,0.104,0.088,0.061  …  0.026,0.026,0.04,0.04,0.022,0.022,0.068,0.097,0.104,0.009],[0.461,0.057,0.023,0.034,0.052,0.032,0.0,0.028,0.024,2.459  …  0.055,0.055,0.083,0.083,0.046,0.046,0.142,0.203,0.218,0.0],
73x73 sparse matrix with 289 Float64 entries:
	[1 ,  1]  =  87.9326
	[2 ,  1]  =  -71.4286
	[3 ,  1]  =  -4.73934
	[5 ,  1]  =  -11.7647
	[1 ,  2]  =  -71.4286
	[2 ,  2]  =  84.5109
	[4 ,  2]  =  -7.87402
	[6 ,  2]  =  -5.20833
	[1 ,  3]  =  -4.73934
	[3 ,  3]  =  25.0475
	⋮
	[60, 71]  =  -10.3093
	[61, 71]  =  -11.4943
	[68, 71]  =  -90.9091
	[71, 71]  =  223.824
	[73, 71]  =  -111.111
	[51, 72]  =  -11.9048
	[63, 72]  =  -19.2308
	[72, 72]  =  31.1355
	[21, 73]  =  -10.3093
	[71, 73]  =  -111.111
	[73

## Scale $\theta_i$ to obtain norm constraint

The form of the trust region subproblem requires the quadratic constraint to take the form of a norm constraint.

To replace our quadratic constraint with a norm constraint, we introduce one additional variable per time step called $\hat{\theta}_{ik}^{(t)}$. This variable is defined in terms of the angle variables associated with the endpoints of the chosen line $(i,k)$:

\begin{align}
\hat{\theta}_{ik}^{(t)} &= \tau^{\frac{1}{2}(T-t)}\left(\theta_i^{(t)} - \theta_k^{(t)}\right) \\
\implies - \tau^{\frac{1}{2}(T-t)}(\theta_i) + \tau^{\frac{1}{2}(T-t)}(\theta_k) + \hat{\theta}_{ik}^{(t)} &= 0
\end{align}

With the new variables $\hat{\theta}_{ik}$ defined in this way, we can replace $$z^\top Q_\theta z=c$$ by $$\lVert \hat{\theta}_{ik}\rVert=c~.$$ This is the norm constraint we desire.

We extend the vector of variables $z$ to include these $T$ additional variables, and we augment $A$ and $b$ to reflect the above relationship.

In [5]:
function tmp_inst_A_scale(n,Ridx,T,tau,slack,line)
    """ Augment A with T additional rows relating
    angle difference variables to angle variables.
    
    Returns a T-by-(n+nr+1)*T matrix that may be
    concatenated with the output of temp_inst_A
    
    Arguments:    
    * n is the number of nodes in the network
    * Ridx is a vector indicating wind nodes
    * T is the number of time steps
    * tau is the thermal coefficient from IEEE 738
    * slack is the index of the slack bus
    * line is the pair (i,k) indicating the chosen
    line
    """
    (i,k) = line
    nr = length(Ridx)
    
    A = zeros(T,(nr+n+2)*T)
    
    for t = 1:T
        i_pos = (nr+n+1)*(t-1) + n + i
        k_pos = (nr+n+1)*(t-1) + n + k
        coef = tau^((1/2)*(T-t))
        A[t,i_pos] = -coef
        A[t,k_pos] = coef
        A[t,(n+nr+1)*T + t] = 1
    end
    slack_cols = [(nr+n+1)*(t-1) + nr + slack for t in 1:T]
    
    # remove slack columns:
    return sparse(A[:,setdiff(1:(n+nr+2)*T,slack_cols)])
end

tmp_inst_A_scale (generic function with 1 method)

In [11]:
n = 3
Ridx = [1,2,3]
T = 2
tau = 2
slack = 3
line = (1,2)

A2 = tmp_inst_A_scale(n,Ridx,T,tau,slack,line)
full(A2)

2x14 Array{Float64,2}:
 0.0  0.0  0.0  -1.41421  1.41421  0.0  …  0.0  0.0   0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0   0.0      0.0      0.0     0.0  0.0  -1.0  1.0  0.0  0.0  1.0

## Making $Q_\theta$

$Q_\theta$ is an identity matrix in the lower-right corner with zeroes everywhere else.

In [17]:
function tmp_inst_Qtheta(n,nr,T,tau)
    """ Generate Q_theta in the temperature constraint
    of a temporal instanton problem instance.
    "line" has the form (i,k), where i and k refer to
    the endpoints of the chosen line.
    """
    Qtheta = zeros((nr+n+1)*T,(nr+n+1)*T)
    
    Qtheta[end-T+1:end,end-T+1:end] = eye(T)
    return sparse(Qtheta)
end

tmp_inst_Qtheta (generic function with 2 methods)

In [20]:
Q = tmp_inst_Qtheta(3,2,3,2)
full(Q)

18x18 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 